In [ ]:
import torch
import numpy as np
from torch import nn
from scipy.sparse import csr_matrix

# https://stackoverflow.com/questions/52299420/scipy-csr-matrix-understand-indptr

In [ ]:
class myGraphSAINT(nn.Module):
    def __init__(self):
        super(myGraphSAINT, self).__init__()
    
    def forward(self, x):
        return x
    
    def sampleGraph(self, graph):
        pass

In [ ]:
x = torch.ones(3)
model = myGraphSAINT()
print(model(x))

In [ ]:
adj_train = np.load("./ppi/adj_train.npz")

data = adj_train["data"]
indices = adj_train["indices"]
indptr = adj_train["indptr"]
shape = adj_train["shape"]

adj_matrix = csr_matrix((data, indices, indptr), shape=shape).toarray()
print(adj_matrix.shape)